In [21]:
import pandas as pd
import urllib
import json
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

def handler(event,context,testURL):
    
    # For testing
    df = pd.read_csv(testURL)
        
    # FOR prod
    # original_xl = urllib.request.urlopen(json.loads(event['body'])['url'])
    # df = pd.read_excel(original_xl, sheetname=None, header=3)
    
    result = {'worksheets': []}

    # Columns to retain/rename from IO export
    colsToRetain = ['Order', #convert to 'Internal Order'
                    'Val_in_rep_cur_', #convert to 'Settlement Amount'
                    'classification_names', #convert to 'Asset Class'
                    'Purchase_order_text', #convert to 'Asset description'
                    'CoCd', #convert to "Company Code"
                    'Site',
                    'Transaction_Grouping',
                    'Serial_number',
                    'Depreciation_Key___Area_01',
                    'Planned_useful_life_in_years_Book_01'
                   ]

    # Retain above and delete the rest
    df.drop(df.columns.difference(colsToRetain), 1, inplace=True)

    # Rename columns for ERP
    df.rename(columns={'Order':'Internal Order',
                       'Val_in_rep_cur_':'Settlement Amount',
                       'classification_names':'Asset Class',
                       'Purchase_order_text':'Asset description',
                       'CoCd':'Company Code',
                       'Transaction_Grouping':'Transaction Grouping',
                       'Serial_number':'Serial number',
                       'Depreciation_Key___Area_01':'Depreciation Key - Area 01',
                       'Planned_useful_life_in_years_Book_01':'Planned useful life in years Book 01'
                      },inplace=True)

    # Columns to add
    colsToAdd = ['Asset Main No.',
                'Asset Sub No.',
                'Main Asset No',
                'Create Sub Asset',
                'Inventory Number',
                #'Internal Order',
                #'Serial number',
                'Asset is Managed Historically',
                'Additional asset description',
                'Asset capitalisation date',
                'Cost Centre',
                #'Internal Order',
                'Number of Similar Assets',
                'Ev. 1',
                'Ev. 2',
                'Ev. 3',
                'Ev. 4',
                'Ev. 5',
                'Investment Order',
                'Vendor',
                'Supplier Name',
                'Manufacturer of asset',
                'License plate number',
                'Original asset that was transferred',
                'Original (Sub) asset that was transferred',
                'Inventory No.',
                'Original asset that was transferred',
                'Original (Sub) asset that was transferred',
                'Deactivate Dep Area 01',
                #'Depreciation Key - Area 01',
                #'Planned useful life in years Book 01',
                'Planned useful life in periods Book 01',
                'Deactivate Dep Area 15',
                'Depreciation Key - Area 15',
                'Planned useful life in years Book 15',
                'Planned useful life in periods Book 15',
                'Deactivate Dep Area 30',
                'Depreciation Key - Area 30',
                'Planned useful life in years Book 30',
                'Planned useful life in periods Book 30',
                ]


    # Add new columns
    df[colsToAdd] = pd.DataFrame([len(colsToAdd)*['']], index=df.index)

    # Update columns
    df['Number of Similar Assets']=1
    df['Cost Centre'] = df['Site']
    df['Asset is Managed Historically']='X'
    #df.insert(0,'Transaction Grouping',df.index)
    
    # Rearrange Columns
    cols = ['Transaction Grouping','Internal Order','Asset Main No.','Asset Sub No.','Settlement Amount','Main Asset No','Create Sub Asset','Asset Class','Company Code','Number of Similar Assets','Asset description','Additional asset description','Serial number','Inventory Number','Asset is Managed Historically','Asset capitalisation date','Cost Centre','Internal Order','Site','Ev. 1','Ev. 2','Ev. 3','Ev. 4','Ev. 5','Investment Order','Vendor','Supplier Name','Manufacturer of asset','License plate number','Original asset that was transferred','Original (Sub) asset that was transferred','Inventory No.','Original asset that was transferred','Original (Sub) asset that was transferred','Deactivate Dep Area 01','Depreciation Key - Area 01','Planned useful life in years Book 01','Planned useful life in periods Book 01','Deactivate Dep Area 15','Depreciation Key - Area 15','Planned useful life in years Book 15','Planned useful life in periods Book 15','Deactivate Dep Area 30','Depreciation Key - Area 30','Planned useful life in years Book 30','Planned useful life in periods Book 30']
    df = df[cols]

    # Clear NAs
    df = df.fillna(value='')

    #FOR TESTING - csv write
    outputFile = "/Volumes/GoogleDrive/My Drive/Enterprise digital/Clients/7-Eleven/4. 7 Eleven - Deliverables/SAP Custom Export/4185RETANK_Output.csv"
    df.to_csv(index=False,path_or_buf=outputFile)

    df_dict = df.to_dict('split')
    result['worksheets'].append({'name': 'seven_eleven_SAP', 'data': [df_dict['columns']] + df_dict['data']})

    return {
        'statusCode': 200,
        'body': json.dumps(result) if result else json.dumps({}),
        'headers': {
            'Content-Type': 'application/json',
        },
    }

In [22]:
handler(1,1,"/Volumes/GoogleDrive/My Drive/Enterprise digital/Clients/7-Eleven/4. 7 Eleven - Deliverables/SAP Custom Export/4185RETANK_Input.csv")

                           Asset description Company Code  Internal Order  \
0  ARCHITECH AND ENGINEERING SERVICES-RETANK         SEPL          126042   
1  ARCHITECH AND ENGINEERING SERVICES-RETANK         SEPL          126042   
2  ARCHITECH AND ENGINEERING SERVICES-RETANK         SEPL          126042   

   Site Settlement Amount  Transaction Grouping  Serial number  \
0  4185            540.00                     1            NaN   
1  4185          7,350.00                     1            NaN   
2  4185            540.00                     1            NaN   

  Depreciation Key - Area 01  Planned useful life in years Book 01  \
0                        NaN                                   NaN   
1                        NaN                                   NaN   
2                        NaN                                   NaN   

   Asset Class  
0        30500  
1        30500  
2        30500  


/Users/jjackaeast001/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:117: UserWarning: DataFrame columns are not unique, some columns will be omitted.


{'statusCode': 200,
 'body': '{"worksheets": [{"name": "seven_eleven_SAP", "data": [["Transaction Grouping", "Internal Order", "Asset Main No.", "Asset Sub No.", "Settlement Amount", "Main Asset No", "Create Sub Asset", "Asset Class", "Company Code", "Number of Similar Assets", "Asset description", "Additional asset description", "Serial number", "Inventory Number", "Asset is Managed Historically", "Asset capitalisation date", "Cost Centre", "Internal Order", "Site", "Ev. 1", "Ev. 2", "Ev. 3", "Ev. 4", "Ev. 5", "Investment Order", "Vendor", "Supplier Name", "Manufacturer of asset", "License plate number", "Original asset that was transferred", "Original (Sub) asset that was transferred", "Inventory No.", "Original asset that was transferred", "Original (Sub) asset that was transferred", "Deactivate Dep Area 01", "Depreciation Key - Area 01", "Planned useful life in years Book 01", "Planned useful life in periods Book 01", "Deactivate Dep Area 15", "Depreciation Key - Area 15", "Planned